# Process

**Zain Nomani 2202455**

## Import data

In [1]:
import pandas as pd
X: pd.DataFrame = pd.read_csv("data/Train/train_x.csv", header = None)
y: pd.DataFrame = pd.read_csv("data/Train/train_y.csv", header = None)
X_test: pd.DataFrame = pd.read_csv("data/Test/test_x.csv", header = None)
y_test: pd.DataFrame = pd.read_csv("data/Test/test_y.csv", header = None)

---
## Task A

Implement regression tree with stop criterion being "minimum number of samples required to split a node" i.e. if node has less samples than a value of min_samples_split then the splitting of the node must stop. Implement from scratch and check performance with Scikit-Learn

### RegressionTreeNode class

In [2]:
"""
Node class for regression classifier
Represents decision stump
Builds classification down to min_samples_split=2
Constructs entire tree from current node
Testing capabilities included
"""
import pandas as pd
import numpy as np

class RegressionTreeNode:
    """
    Parameters:
        - X: pd.DataFrame: Feature vectors assigned to stump
        - y: pd.DataFrame: Results of each feature vector assigned to stump
        - min_samples_split: int: Hyperparameter used to control complexity
        - feature: int: feature used to split data
        - threshold: float: point at which data is split on the feature
        - samples: int: number of samples assigned to stump
        - right_child: Node: Child node assigned X_right and y_right
        - left_child: Node: Child node assigned X_left and y_left
        - parent: Node: Node for which this self node is either a right_child or left_child
    
    Public Methods:    
        - summarise(): Prints key values of this stump
        - train(): Recursively calls _singleRun() to split current and all child nodes
        - predict(): Predicts the output of a single feature vector by traversing down tree
        - test(): Given multiple feature vectors, calls predict() on each and calculates Error
    
    Private Methods:
        - _singleRun(): NumPy methods to calculate best split point of current node
    
    Helpful Methods:
        - Leaf(): Is the current node a leaf or not?
        - Feature(): Returns feature to split on
        - Threshold(): Returns threshold value on which data is split
        - MSE(): Returns mean squared error of self.y from mean
        - Value(): Mean value of self.y on this stump
        - __hash__() and __eq__: Hashing for dictionary memoization when recursively running
    
    """
    def __init__(self):
        self.X = None
        self.y = None
        self.feature = self.threshold = self.right_child = self.left_child = self.parent = None # Feature to be split upon based on X
        self.min_samples_split = None
        self.leftIndex=None
        self.rightIndex=None

    """Public Methods"""
    
    def summarise(self):
        print("Feature: ", self.Feature())
        print("Threshold: ", self.Threshold())
        print("Samples: ", self.Samples())
        print("Value: ", self.Value())
    
    def train(self, X: pd.DataFrame, y: pd.DataFrame, min_samples_split: int = None, memo: dict = None) -> "RegressionTreeNode":
        """Recursively call _singleRun()"""
        if len(X) != len(y):
            raise ValueError("Input and output vector mismatch")
        if memo is None:
            memo = {}
        Xcopy = X.copy()
        ycopy = y.copy()
        if 'tmp' in Xcopy.columns: # Remove duplicates for ease of calculation
            Xcopy = Xcopy.loc[:, X.columns != 'tmp']
        Xcopy.insert(len(X.columns), 'tmp', y)
        Xcopy = Xcopy.drop_duplicates(keep='first')
        ycopy = Xcopy.pop('tmp')
        self.X, self.y = Xcopy, ycopy
        if min_samples_split is None:
            min_samples_split = 2
        self.min_samples_split = min_samples_split
        if self in memo:
            return memo[self]
        self = self._singleRun()
        memo[self] = self
        if self.leftIndex is not None and self.rightIndex is not None:
            self.left_child = RegressionTreeNode().train(self.X[self.leftIndex], self.y[self.leftIndex], min_samples_split, memo)
            self.right_child = RegressionTreeNode().train(self.X[self.rightIndex], self.y[self.rightIndex], min_samples_split, memo)
        return self
    
    def predict(self, feature_vector: pd.Series, hyperParams: float = None) -> float:
        if hyperParams is None:
            hyperParams = self.min_samples_split
        if hyperParams > self.Samples() or self.Feature() is None:
            return self.Value()
        elif feature_vector[self.Feature()] <= self.Threshold():
            return self.left_child.predict(feature_vector, hyperParams)
        elif feature_vector[self.Feature()] > self.Threshold():
            return self.right_child.predict(feature_vector, hyperParams)
        else:
            return None
    
    def test(self, feature_vectors: pd.DataFrame, ground_truth: pd.DataFrame, hyperParams: int = None) -> float:
        if hyperParams is None:
            hyperParams = self.min_samples_split
        if len(feature_vectors) != len(ground_truth):
            raise ValueError(f"There are {len(feature_vectors)} Feature Vectors but {len(ground_truth)} Ground Truths")
        # Output of feature vectors should be NumPy arrays
        groundTruth: np.ndarray = ground_truth.to_numpy() # For speed of calculation
        predictions: np.ndarray = feature_vectors.apply(lambda x: self.predict(x, hyperParams), axis=1).to_numpy().reshape(-1,1)
        return np.nanmean((predictions - groundTruth)**2)
    
    """Private methods"""
    
    def _singleRun(self) -> "RegressionTreeNode":
        """Evaluate a single tree to find optimal split point. Create children"""
        if self.Samples() >= self.min_samples_split:
            X_vals = self.X.to_numpy().transpose()
            y_vals = self.y.to_numpy().transpose()
            # Choose 100 values from each feature
            # Allows us to create a NumPy array for efficiency
            thresholds: np.ndarray = np.linspace(X_vals.min(axis=1), X_vals.max(axis=1), 100).transpose()
            result = X_vals[:, np.newaxis, :] <= thresholds[:, :, np.newaxis]
            inverse = X_vals[:, np.newaxis, :] > thresholds[:, :, np.newaxis]
            # Turn any falses into np.nan to remove them from the multiplication with y_vals
            # i.e. Any rows that belong to the other child are turned into np.nan
            left = np.where(result.astype(float) == 0, np.nan, result) * y_vals
            right = np.where(inverse.astype(float) == 0, np.nan, inverse)*y_vals
            mean_left = np.nanmean(left, axis=-1)
            mean_right =np.nanmean(right,axis=-1)
            mse_left = np.nanmean((left - mean_left[:,:,np.newaxis])**2, axis=-1)
            mse_right=np.nanmean((right-mean_right[:, :,np.newaxis])**2, axis=-1)
            if len(y_vals) == 0:
                return ValueError("Length of assigned dataframe is 0")
            factorLeft = np.sum(~np.isnan(left), axis=-1) / len(y_vals)
            factorRight = np.sum(~np.isnan(right), axis=-1)/len(y_vals)

            loss = factorLeft*mse_left + factorRight*mse_right # Weighted MSE
            min = np.nanmin(loss)
            argmin = np.where(loss == min)

            self.feature = argmin[0][0]
            self.threshold = thresholds[argmin[0][0]][argmin[1][0]]

            left_indices = result[argmin[0][0]][argmin[1][0]] # Rows of X and y which go to left child
            right_indices = ~left_indices
            
            self.leftIndex = left_indices
            self.rightIndex = right_indices
                
        return self
    
    """Helpful methods"""

    def __hash__(self):
        return hash(tuple(self.y.index))
    
    def __eq__(self, other):
        if isinstance(other, RegressionTreeNode):
            return self.y.index == other.y.index
        return False
    
    def Leaf(self) -> bool:
        return True if (self.right_child is None and self.left_child is None) else False
    
    def Feature(self) -> int:
        return self.feature
    
    def Threshold(self) -> float:
        return self.threshold

    def MSE(self) -> float | None:
        return np.mean((self.y.values - np.mean(self.y))**2) if self.samples > 0 else None

    def Samples(self) -> int:
        if self.y is None or self.X is None:
            return None
        elif len(self.y) != len(self.X):
            return None
        else:
            return len(self.y)
    
    def Value(self) -> float:
        if len(self.y) == 0:
            return None
        return self.y.mean()

### Performance of RegressionTreeNode

In [3]:
import warnings
warnings.filterwarnings("ignore", message="Mean of empty slice")
tree = RegressionTreeNode().train(X, y, 2)
print("RegressionTreeNode MSE on training data: ", tree.test(X, y))
print("RegressionTreeNode MSE on test data: ", tree.test(X_test, y_test))

RegressionTreeNode MSE on training data:  0.0
RegressionTreeNode MSE on test data:  0.7092307692307692


### Testing performance vs Sklearn

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

sklearn_tree = DecisionTreeRegressor()
sklearn_tree.fit(X, y)
sklearn_test_predictions = sklearn_tree.predict(X_test)
sklearn_train_predictions = sklearn_tree.predict(X)
mseTrain = mean_squared_error(y, sklearn_train_predictions)
mseTest = mean_squared_error(y_test, sklearn_test_predictions)

print("Sklearn MSE on training data: ", mseTrain)
print("Sklearn MSE on test data: ", mseTest)

Sklearn MSE on training data:  0.0
Sklearn MSE on test data:  0.7384615384615385


---
## Task 2

Second Task: Using regression tree, fit an ensemble of exactly 5 trees. After training, compute performance on test data, and avoid overfitting by using regularisation techniques to limit complexity of each tree. Use Minimum Samples Split regularisation technique: Hyper-parameter that ensures a node has minimum number of samples before it can be split to prevent tiny splits that create noise. Note: You can set each tree's complexity different to another

### RegressionTreeKFold

In [6]:
""" 
Regression Tree Classifier
Builds regression classifier down to min_samples_split=2
Can predict using any min_samples_split
Wraps around RegressionTreeNode
Identifies best min_samples_split using K-fold cross validation
"""
import pandas as pd
import numpy as np

class RegressionTreeKFold:
    """
    Parameters:
        - X: pd.DataFrame: Feature vectors as input
        - y: pd.DataFrame: Corresponding results of each feature vector
        - splits: list[np.ndarray]: List of k lists, each containing indices of samples in each fold
        - hyperParams: list[int]: List of hyperparameters to test
        - validation: np.ndarray: Initial 3D NumPy array of each fold. Contains boolean indexing for filtering X and y
        - errors: np.ndarray: 2D NumPy Array of Mean Square Errors for each fold and hyperparameter
        - metrics: np.ndarray: Taking mean of errors: 1D NumPy array of MSE for each hyperParameter
        - best: float: Best hyperParameter which minimises the MSE. Found by argmin(metrics)
        - root: Node: Root Node of regression tree
    
    Public Methods:
        - train(): Builds regression tree on each fold and calls predict() on each hyperparameter
        - predict(): Calls node.predict(vector, hyperParam). Predicts with best value if not specified
        - test(): Tests given feature vectors on ground truth and provided hyperParameter - outputs MSE
    
    Private Methods:
        - _makeFolds(): Builds self.splits to create k folds for cross validation
        - _getParams(): Constructs list of hyperparameters to test
        - _setup(): Build self.validation: 3D array of booleans to split dataframes for training and testing
        - _findBest(): Using self.validation, build self.errors by iterating using for loop to test each fold
    """
    def __init__(self): # Initialise all parameters to None
        self.X = self.y = self.splits = self.errors = self.metrics = self.best = self.root = None
        self.hyperParams: list[int] = None
        self.validation = None
    
    """Public Methods"""
    
    def train(self, X, y):
        self.X, self.y = X, y
        self._makeFolds()
        # print("Folds created")
        self._getParams()
        # print("HyperParameters set up")
        self._setup()
        # print("Matrices set up. Now running...")
        self._findBest()
        # print("Process complete. Now training final regression tree with param ", self.best)
        self.root = RegressionTreeNode().train(self.X, self.y, self.best)
        return self
    
    def predict(self, feature_vector: pd.Series, hyperParams: float = None) -> float:
        if self.root is None:
            return "Root is undefined"
        if hyperParams is None:
            hyperParams = self.best
        return self.root.predict(feature_vector, hyperParams)

    def test(self, feature_vectors: pd.DataFrame, ground_truth: pd.DataFrame, hyperParams: int = None) -> float:
        if self.root is None:
            return "Root is undefined"
        if hyperParams is None:
            hyperParams = self.best
        return self.root.test(feature_vectors, ground_truth, hyperParams)

    """Private Methods"""

    def _makeFolds(self) -> list[np.ndarray]:
        """
        Select indices in each set.
        """
        if self.y is None:
            return ValueError("Data undefined")
        length: int = len(self.y)
        order: np.ndarray = np.arange(0, length)
        splits: list[np.ndarray] = np.array_split(order, 5)
        self.splits = splits
        return splits
    
    def _getParams(self):
        """Create list of hyperparameters to test"""
        if len(self.X) != len(self.y) or len(self.X) == 0:
            raise ValueError("Improper data")
        self.hyperParams = [i for i in range(2,len(self.X), 10)]
        return self
    
    def _setup(self) -> np.ndarray:
        """
        Set up self.validation to begin running
        """
        if self.splits is None:
            return ValueError("Data not yet split into folds")
        if self.y is None or self.X is None:
            return ValueError("Data not yet defined")
        if not isinstance(self.hyperParams, list):
            return TypeError("HyperParams needs to be of type list[int]")
        row_vector: np.ndarray = np.zeros((len(self.splits), len(self.y)), dtype=bool)
        for i in range(len(self.splits)):
            validation: np.ndarray = self.splits[i]
            row_vector[i][validation] = True
        row_vector = np.repeat(row_vector[np.newaxis, :, :], len(self.hyperParams), axis=0)
        self.validation = row_vector
        return row_vector
    
    def _findBest(self):
        """Compute errors of each HyperParameter"""
        errors = np.zeros((self.validation.shape[0], self.validation.shape[1]), dtype=float)
        validation = np.transpose(self.validation, (1, 0, 2))
        # print("Validation Shape post tranposing: ", validation.shape)
        # Shape should by (#folds, #hyperparams, 5197)
        for i in range(validation.shape[0]): # For each fold
            # print("Testing fold ", i+1)
            # Rows are split by fold i.e. validation[i][j] = validation[i][k] since we expand self.validation by the number of folds we are measuring
            node = RegressionTreeNode().train(self.X[~validation[i][0]], self.y[~validation[i][0]], 2)
            for j in range(validation.shape[1]): # For each hyperParam
                # print("HyperParam ", self.hyperParams[j])
                # node = RegressionTreeNode().train(self.X[~validation[i][j]], self.y[~validation[i][j]], self.hyperParams[j])
                error = node.test(self.X[validation[i][j]], self.y[validation[i][j]], self.hyperParams[j])
                # print("Hyperparam: ", self.hyperParams[j], " Error: ", error)
                errors[j][i] = error

        self.errors = errors
        # errors is the MSE of each fold for each hyperparam
        metrics = np.mean(errors, axis=-1) # metrics gives the MSE for each hyperparam
        self.metrics = metrics
        best = np.argmin(metrics)
        self.best = self.hyperParams[best]

### RegressionTreeEnsemble

In [7]:
"""
Ensemble class to boost Regression Tree
Uses Forward Stagewise Additive Modelling
"""
import pandas as pd
import numpy as np

class RegressionTreeEnsemble:
    """
    Parameters:
        - trees: Ensemble of 5 RegressionTreeKFold
    
    Public Methods:
        - train(X, y): Iteratively calculate the residuals between the ground truth and sum of all previous trees (up to 5)
        - predict(X): Run predict on X over entire ensemble
        - test(X, y): Predict all X and return MSE between prediction and ground truth y
    """
    def __init__(self):
        self.trees: list[RegressionTreeKFold] = None

    def train(self, X, y):
        trees = []
        residuals = y.to_numpy()

        for i in range(1, 6):  # 5 trees
            # print(f"Training tree {i}")
            tree = RegressionTreeKFold().train(X, pd.DataFrame(residuals))
            # print(f"Best min_samples_split for tree {i}: {tree.best}")
            trees.append(tree)
            cumulative_predictions = sum(X.apply(t.predict, axis=1).to_numpy() for t in trees)
            residuals = y.to_numpy() - cumulative_predictions.reshape(-1, 1)

        self.trees = trees
        return self

    def predict(self, X):
        return sum(X.apply(tree.predict, axis=1) for tree in self.trees)
    
    def test(self, X_test: pd.DataFrame, y_test:pd.DataFrame):
        result = self.predict(X_test)
        mse = (result.to_numpy()[:, np.newaxis] - y_test.to_numpy())**2
        return np.mean(mse)

### Run Ensemble on test data

In [8]:
import warnings
warnings.filterwarnings("ignore", message="Mean of empty slice")
ensemble = RegressionTreeEnsemble().train(X, y)
print("MSE of custom ensemble on training data: ", ensemble.test(X, y))
print("MSE of custom ensemble on test data: ", ensemble.test(X_test, y_test))

MSE of custom ensemble on training data:  0.4019919200117535
MSE of custom ensemble on test data:  0.5281653087508057


### Performance of Scikit-Learn

In [9]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Training here
n_trees: int = 5
predictions: np.ndarray = np.zeros(y.shape)
models: list[DecisionTreeRegressor] = []

for _ in range(n_trees):
    residuals = y - predictions
    # Define hyperparameters the same as are defined in RegressionTreeKFold: 2 to 5197 with intervals of 10
    param_grid: dict = {'min_samples_split': [i for i in range(2, 5197, 10)]}
    tree = DecisionTreeRegressor(random_state=1)
    grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X, residuals)
    best_tree = grid_search.best_estimator_
    models.append(best_tree)
    predictions += best_tree.predict(X).reshape(-1,1)

# models is our ensemble of trees      

training_predictions = np.zeros(y.shape)
for model in models:
    training_predictions += model.predict(X).reshape(-1,1)
test_predictions = np.zeros(y_test.shape)
for model in models:
    test_predictions += model.predict(X_test).reshape(-1,1)

mseTraining = mean_squared_error(y, training_predictions)
mseTest = mean_squared_error(y_test, test_predictions)

print("Mean Squared Error of ensemble on training set: ", mseTraining)
print("Mean Squared Error of ensemble on test set: ", mseTest)

Mean Squared Error of ensemble on training set:  0.40980966341572894
Mean Squared Error of ensemble on test set:  0.5267987468000692
